# Lexicon Lablling
using VADER and TextBlob to predict the polarity of tweets and determine the overall sentiment polarity ('positive', 'negative', or 'neutral') of each text by comparing the VADER and TextBlob sentiment scores. And, if there is any suspicious result from VADER and TextBlob for a tweet, the code will write a question mark '?' to the polarity so that we can double check it again.

In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import pandas as pd
import nltk

# read the .csv file for sentiment analysis
df = pd.read_csv('traintext.csv')

#set the initial number
df['textblob'] = 999
df['vader'] = 999
df['polarity'] = 'N'

# initialize VADER and TextBlob sentiment analyzers
nltk.download('vader_lexicon') # this is for VADER
vader = SentimentIntensityAnalyzer()

# VADER sentiment analysis
df['vader'] = df['text'].apply(lambda x: 
                               vader.polarity_scores(str(x))['compound'])

# TextBlob sentiment analysis
df['textblob'] = df['text'].apply(lambda x: 
                                  TextBlob(str(x)).sentiment.polarity)


# determining the polarity
df['polarity'] = '?'
df.loc[(df['vader'] > 0) & (df['textblob'] > 0), 'polarity'] = 2 #'POSITIVE'
df.loc[(df['vader'] < 0) & (df['textblob'] < 0), 'polarity'] = 0 #'NEGATIVE'
df.loc[(df['vader'] == 0) & (df['textblob'] == 0), 'polarity'] =1 #'NEUTRAL'
df.loc[(df['vader'] + df['textblob'] > 0.6), 'polarity'] =2 #'Postive'
df.loc[((df['vader'] + df['textblob'] > 0.0) & (df['vader'] + df['textblob'] < 0.6)  ), 'polarity'] =1 #'Neutral'
# print the results
df

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\andy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Unnamed: 0,rating,text,textblob,vader,polarity
0,0,4,hey yum sangria snorkel sound amaz glad fun,0.4,0.7430,2
1,1,4,enjoy sunday dinner,0.4,0.4939,2
2,2,4,show awesom vega u make last night tour memor,0.0,0.0000,1
3,3,4,damon welcom twitter nation,0.0,0.0000,1
4,4,0,amp think find mine leav guy thought well,0.0,0.2732,1
...,...,...,...,...,...,...
9995,9995,4,book 8 love stake come march 2010 star robbi m...,0.5,0.6369,2
9996,9996,0,ca believ get 4 day leav love school wonder fr...,0.5,0.8126,2
9997,9997,0,alway look like freak alway put ear top see he...,0.5,0.1027,2
9998,9998,4,hanki never abl watch movi madagascar way hehe,0.0,0.0000,1


>> ### another option: to remove doubtful and unsure tweets

In [8]:
df =df[df['polarity'] != '?']

### Save to file

In [17]:
df.to_csv('pred.csv', index=False)